# ConversationKGMemory

지식 그래프의 힘을 활용하여 정보를 저장하고 불러옵니다.

이를 통해 모델이 서로 다른 개체 간의 관계를 이해하는 데 도움을 주고, 복잡한 연결망과 역사적 맥락을 기반으로 대응하는 능력을 향상시킵니다.

## history -> 문장 전체 저장, 의미를 구조적으로 이해하지 못함 = 문장이 어떤 관계를 갖는지 인식하지 못함
## KG -> 문장 내부의 구조적 관계와 의미를 저장 (subject, relation, object 등등)


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import json

c:\Users\jeffr\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-SGRq4Tjy-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

kg_store = []

# 3) 매우 단순한 KG 추출기
def extract_triplet(text: str): #무엇을 기억할지 결정하는 규칙
    
    #문장에서 (주어,관계,객체) 추출 – 구버전 KGMemory 기능의 최소 대체.
    #복잡한 NLP 사용하지 않고 코드가 원하는 기능만 수행.
    
    triples = []

    # Pangyo 에 거주중인 김셜리
    if "거주" in text and "김셜리" in text:
        triples.append({"subject": "김셜리", "relation": "거주지", "object": "Pangyo"})

    # 신입 디자이너
    if "신입 디자이너" in text and "김셜리" in text:
        triples.append({"subject": "김셜리", "relation": "직업", "object": "신입 디자이너"})

    return triples


def save_context(input, output):
    #ConversationKGMemory.save_context 대체
    text = input.get("input", "")
    triples = extract_triplet(text)
    for t in triples:
        kg_store.append(t)


def load_memory_variables(query: dict):
    #ConversationKGMemory.load_memory_variables 대체
    # 단순히 KG 전체를 반환 (구버전의 반환 구조와 동일)
    return {"history": json.dumps(kg_store, ensure_ascii=False)}


save_context( # 전에 대체되었던 sav_context는 메소드고 이건 함수
    {"input": "이쪽은 Pangyo 에 거주중인 김셜리씨 입니다."},
    {"output": "김셜리씨는 누구시죠?"}
)

save_context(
    {"input": "김셜리씨는 우리 회사의 신입 디자이너입니다."},
    {"output": "만나서 반갑습니다."}
)


In [4]:
result = load_memory_variables({"input": "김셜리씨는 누구입니까?"})

print("----- 대화 지식 그래프 메모리 -----")
print(result)

----- 대화 지식 그래프 메모리 -----
{'history': '[{"subject": "김셜리", "relation": "거주지", "object": "Pangyo"}, {"subject": "김셜리", "relation": "직업", "object": "신입 디자이너"}]'}


## Chain 에 메모리 활용하기

`ConversationChain` 에 `ConversationKGMemory` 를 메모리로 지정하여 대화를 나눈 후 memory 를 확인해 보도록 하겠습니다.


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

template = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. 
The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""

# 3. 최신 버전 Prompt 형태로 변환
prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# 4. history 메모리 (ConversationKGMemory 대체)
history = InMemoryChatMessageHistory()

# 5. ConversationChain 대체
conversation_with_kg = RunnableWithMessageHistory(
    prompt | llm,
    lambda session_id: history,
    input_messages_key="input",
    history_messages_key="history"
)

# 사용 예시

user_question = "김셜리씨는 누구입니까?"

response = conversation_with_kg.invoke(
    {"input": user_question},
    config={"configurable": {"session_id": "session1"}}
)

print(response.content)
print(history.messages)

죄송하지만, 제공된 정보에는 김셜리씨에 대한 내용이 없습니다. 김셜리씨에 대해 더 알고 싶으시면 다른 출처를 참고해 보시기 바랍니다.
[HumanMessage(content='김셜리씨는 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='죄송하지만, 제공된 정보에는 김셜리씨에 대한 내용이 없습니다. 김셜리씨에 대해 더 알고 싶으시면 다른 출처를 참고해 보시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 110, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcVO2vsgGYuWBi0BwyVPyIEO0YeX9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ee2a92d5-7406-45a7-9bf1-c02502416b80-0', usage_metadata={'input_tokens': 110, 'output_tokens': 40, 'total_tokens': 150, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

첫 번째 대화를 시작합니다. 간단한 인물에 대한 정보를 제공해 보겠습니다.


In [ ]:
# conversation_with_kg.predict(
#     input="My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."
# )

response = conversation_with_kg.invoke(
    {"input": "My name is Teddy. Shirley is a coworker of mine, and she's a new designer at our company."},
    config={"configurable": {"session_id": "session1"}}
)

print(response.content)


[HumanMessage(content='김셜리씨는 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='죄송하지만, 제공된 정보에는 김셜리씨에 대한 내용이 없습니다. 김셜리씨에 대해 더 알고 싶으시면 다른 출처를 참고해 보시기 바랍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 110, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcVO2vsgGYuWBi0BwyVPyIEO0YeX9', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ee2a92d5-7406-45a7-9bf1-c02502416b80-0', usage_metadata={'input_tokens': 110, 'output_tokens': 40, 'total_tokens': 150, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessage(content="My name is T

Shirley 라는 사람에 대한 질문을 진행합니다.


In [9]:
# Shirley 에 대한 질문
# conversation_with_kg.memory.load_memory_variables({"input": "who is Shirley?"})

response = conversation_with_kg.invoke(
    {"input": "Who is Shirley?"},
    config={"configurable": {"session_id": "session1"}}
)
print(response.content)

Shirley is your coworker and a new designer at your company, as you mentioned. If you have any specific questions about Shirley or need assistance related to her, feel free to ask!
